# 🍏 Basic Retrieval-Augmented Generation (RAG) with AIProjectClient 🍎

In this notebook, we'll demonstrate a basic RAG flow using:

- AIProjectClient (ChatCompletions)
- OpenAIClient (Embeddings)
- Azure AI Search (for vector or hybrid search)

Our theme is **Health & Fitness** 🍏 so we’ll create a simple set of health tips, embed them, store them in a search index, then do a query that retrieves relevant tips, and pass them to an LLM to produce a final answer.

>    Disclaimer: This is not medical advice. For real health questions, consult a professional.

# What is RAG?

Retrieval-Augmented Generation (RAG) is a technique where the LLM (Large Language Model) uses relevant retrieved text chunks from your data to craft a final answer. This helps ground the model's response in real data, reducing hallucinations.

<img src="seq-diagrams/3-basic-rag.png" alt="Basic RAG Diagram" />

# 1. Setup

We'll import libraries, load environment variables, and create an AIProjectClient.

>   Complete [2-embeddings.ipynb](2-embeddings.ipynb) notebook before starting this one

In [ ]:
#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Azure.AI.Projects, 1.2.0-beta.5"
#r "nuget: dotenv.net"
#r "nuget: Azure.Search.Documents, 11.7.0"

using System.IO;
using System.ClientModel.Primitives;
using Azure;
using Azure.Core;
using Azure.Identity;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using OpenAI;
using OpenAI.Embeddings;
using OpenAI.Responses;
using OpenAI.Chat;
using Azure.Search.Documents;
using Azure.Search.Documents.Indexes;
using Azure.Search.Documents.Indexes.Models;
using Azure.Search.Documents.Models;
using dotenv.net;

DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".","..", ".env") })); 

In [ ]:
#pragma warning disable OPENAI001

var aiFoundryProjectEndpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
var chatModel = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");
var textEmbeddingModel = Environment.GetEnvironmentVariable("TEXT_EMBEDDING_MODEL");
var searchIndexName = Environment.GetEnvironmentVariable("SEARCH_INDEX_NAME");
var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");

Console.WriteLine($"🔑 Using Tenant ID: {tenantId}");

AIProjectClient projectClient;
OpenAIClient openAIClient;
try
{
    var credential = new DefaultAzureCredential(new DefaultAzureCredentialOptions
    {
        TenantId = tenantId
    });

    // Validate credentials by getting a token
    var tokenRequestContext = new TokenRequestContext(new[] { "https://ai.azure.com/.default" });
    var accessToken = await credential.GetTokenAsync(tokenRequestContext);
    Console.WriteLine("✅ Successfully initialized Azure credentials with correct tenant!");      

    projectClient = new(new Uri(aiFoundryProjectEndpoint), credential);
    Console.WriteLine("🎉 Successfully created AIProjectClient");

    var openAiEndpoint = $"https://{projectClient.OpenAI.Endpoint.Authority.Replace(".services.ai.",".openai.")}/openai/v1";
    Console.WriteLine($"Using OpenAI endpoint: {openAiEndpoint}");
    BearerTokenPolicy tokenPolicy = new(
        credential,
        "https://ai.azure.com/.default");

    openAIClient = new(
        authenticationPolicy: tokenPolicy,
        options: new OpenAIClientOptions()
        {
            Endpoint = new(openAiEndpoint),
        }
    );
    Console.WriteLine("🎉 Successfully created OpenAIClient");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error initializing AIProjectClient: {ex.Message}");
    throw;
}

# 2. Create Sample Health Data

We'll create a few short doc chunks. In a real scenario, you might read from CSV or PDFs, [chunk them up](https://learn.microsoft.com/en-us/azure/search/vector-search-how-to-chunk-documents), embed them, and store them in your search index.

In [ ]:
var healthTips = new[]
{
    new { id = "doc1", content = "Daily 30-minute walks help maintain a healthy weight and reduce stress.", source = "General Fitness" },
    new { id = "doc2", content = "Stay hydrated by drinking 8-10 cups of water per day.", source = "General Fitness" },
    new { id = "doc3", content = "Consistent sleep patterns (7-9 hours) improve muscle recovery.", source = "General Fitness" },
    new { id = "doc4", content = "For cardio endurance, try interval training like HIIT.", source = "Workout Advice" },
    new { id = "doc5", content = "Warm up with dynamic stretches before running to reduce injury risk.", source = "Workout Advice" },
    new { id = "doc6", content = "Balanced diets typically include protein, whole grains, fruits, vegetables, and healthy fats.", source = "Nutrition" }
};

Console.WriteLine("Created a small list of health tips.");

# 3.0. Create or Reset the Index

When creating a vector field in Azure AI Search, the **field definition** must include a vector_search_profile property that points to a matching profile name in your vector search settings.

We'll define a helper function to create (or reset) a vector index with an [HNSW algorithm](https://learn.microsoft.com/en-us/azure/search/vector-search-ranking#algorithms-used-in-vector-search) config.

In [ ]:
async Task CreateHealthTipsIndex(
    string endpoint, string apiKey, string indexName, 
    int dimension=1536 //if using OpenAI text-embedding-3-small
)
{
    var serviceEndpoint = new Uri(endpoint);
    var credential = new AzureKeyCredential(apiKey);
    var indexClient = new SearchIndexClient(serviceEndpoint, credential);

    // Delete the index if it exists
    try
    {
        indexClient.DeleteIndex(indexName);
        Console.WriteLine($"Deleted existing index: {indexName}");
    }
    catch (Exception)
    {
        // Index did not exist
    }

    // Define the vector search configuration
    VectorSearch vectorSearch = new() 
    {
            Profiles = { new VectorSearchProfile("myHnswProfile", "myHnsw") },
            Algorithms = { new HnswAlgorithmConfiguration("myHnsw")  }
    };

    // Define fields
    var fields = new List<SearchField>
    {
        new SimpleField("id", SearchFieldDataType.String) { IsKey = true },
        new SearchableField("content"),
        new SearchableField("source"),
        new SearchField("embedding", SearchFieldDataType.Collection(SearchFieldDataType.Single))
        {
            VectorSearchDimensions = dimension,
            VectorSearchProfileName = "myHnswProfile"
        }
    };

    // Create index definition
    var searchIndexDefinition = new SearchIndex(indexName, fields)
    {
        VectorSearch = vectorSearch
    };

    // Create the index
    var response = await indexClient.CreateIndexAsync(searchIndexDefinition);
    Console.WriteLine($"✅ Created or reset index: {indexName}");
}

# 3.1. Create Index & Upload Health Tips 🏋️

Now we'll put our health tips into action by:

1. **Creating a search connection** to Azure AI Search
2. **Building our index** with vector search capability
3. **Generating embeddings** for each health tip
4. **Uploading the tips** with their embeddings

This creates our knowledge base that we'll search through later. Think of it as building our 'fitness library' that our AI assistant can reference! 📚💪

In [ ]:
// Debug: check the search index 
if (string.IsNullOrEmpty(searchIndexName))
{
    Console.WriteLine("❌ search_index_name is empty or None, setting default...");
    searchIndexName = "healthtips-index";
    Console.WriteLine($"✅ Set search_index_name to: '{searchIndexName}'");
}

In [ ]:
var searchEndpoint = "";
var searchKey = "";
var searchConn = await projectClient.Connections.GetDefaultConnectionAsync(connectionType: ConnectionType.AzureAISearch, includeCredentials: true);
if (searchConn == null)
{
    throw new Exception("❌ No default Azure AI Search connection found!");
}
Console.WriteLine("✅ Got search connection.");

// Debug: Print connection properties to see what's available
Console.WriteLine($"🔍 Connection type: {searchConn.GetType()}");
Console.WriteLine($"🔍 Connection properties: {string.Join(", ", searchConn.GetType().GetProperties().Select(p => p.Name))}");
if (!string.IsNullOrEmpty(searchConn.Target))
{
    Console.WriteLine($"🔍 Connection target: {searchConn.Target}");
    searchEndpoint = searchConn.Target;
    Console.WriteLine("✅ Found Search Endpoint via Target");
}
if (searchConn.Credentials != null)
{
    Console.WriteLine($"🔍 Credentials type: {searchConn.Credentials.GetType()}");
    Console.WriteLine($"🔍 Credentials properties: {string.Join(", ", searchConn.Credentials.GetType().GetProperties().Select(p => p.Name))}");
    searchKey = (searchConn.Credentials as AIProjectConnectionApiKeyCredential)?.ApiKey;
    Console.WriteLine("✅ Found Search Key via Credentials.ApiKey");
}

if (string.IsNullOrEmpty(searchKey))
{
    throw new Exception("❌ Cannot find key in search connection. Please check the connection setup.");
}

Console.WriteLine($"✅ Search endpoint: {searchEndpoint}");

var embeddingsClient = openAIClient.GetEmbeddingClient(textEmbeddingModel);
var sampleDoc = healthTips[0];
var embeddingResponse = await embeddingsClient.GenerateEmbeddingAsync(sampleDoc.content);
var embeddingLength = embeddingResponse.Value.ToFloats().Length;
Console.WriteLine($"✅ Got embedding length: {embeddingLength}");

await CreateHealthTipsIndex(searchEndpoint, searchKey, searchIndexName, dimension: embeddingLength);

var searchClient = new SearchClient(new Uri(searchEndpoint), searchIndexName, new AzureKeyCredential(searchKey));
Console.WriteLine("✅ Created SearchClient.");

var searchDocs = new List<object>();
foreach (var tip in healthTips)
{
    var embeddingResponse = await embeddingsClient.GenerateEmbeddingAsync(tip.content);
    var embeddingVector = embeddingResponse.Value.ToFloats();

    var doc = new
    {
        id = tip.id,
        content = tip.content,
        source = tip.source,
        embedding = embeddingVector
    };
    searchDocs.Add(doc);
}

await searchClient.UploadDocumentsAsync(searchDocs);
Console.WriteLine($"✅ Uploaded {searchDocs.Count} documents to the search index {searchIndexName}.");

# 4. Basic RAG Flow

## 4.1. Retrieve

When a user queries, we:

1. Embed user question.
2. Search vector index with that embedding to get top docs.

## 4.2. Generate answer

We then pass the retrieved docs to the chat model.

>   In a real scenario, you'd have a more advanced approach to chunking & summarizing. We'll keep it simple.

In [ ]:
async Task<string> RagChat(string query, int topK = 3)
{
    // 1. Embed the user query
    var queryEmbeddingResponse = await embeddingsClient.GenerateEmbeddingAsync(query);
    var queryEmbedding = queryEmbeddingResponse.Value.ToFloats();

    // 2. Vector search using Vectorized query
    var vectorizedQuery = new VectorizedQuery(queryEmbedding)
    {
        KNearestNeighborsCount = topK,
        Fields = { "embedding" }
    };
    var searchOptions = new SearchOptions
    {
        VectorSearch = new()
        {
            Queries = { vectorizedQuery }
        },
        Select = { "content", "source" },
    };
    
    SearchResults<SearchDocument> response = await searchClient.SearchAsync<SearchDocument>("*", searchOptions);

    // 3. Collect retrieved documents
    var topDocsContent = new List<string>();
    await foreach (var result in response.GetResultsAsync())
    {
        var content = result.Document["content"].ToString();
        var source = result.Document["source"].ToString();
        topDocsContent.Add($"Source: {source} => {content}");
    }

    // 4.Chat with retrueved docs
    var systemText = @$"You are a health & fitness assistant.
        Answer user questions using ONLY the text from these docs.
        Docs:
         {string.Join("\n", topDocsContent)}
        If unsure, say 'I'm not sure'.";
    
    var chatClient = openAIClient.GetChatClient(chatModel);
    ChatCompletion chatResponse = await chatClient.CompleteChatAsync(messages: new ChatMessage[]
    {
        new SystemChatMessage(systemText),
        new UserChatMessage( query)
    });
    return chatResponse.Content[0].Text;
}

# 5. Try a Query 🎉

Let's do a question about cardio for busy people.

In [ ]:
var userQuery = "What's a good short cardio routine for me if I'm busy?";
var chatResult = await RagChat(userQuery);
Console.WriteLine($"🗣️ User Query: {userQuery}");
Console.WriteLine($"🤖 RAG Answer: {chatResult}");

# 6. Conclusion

We've demonstrated a **basic RAG** pipeline with:

- **Embedding** docs & storing them in **Azure AI Search**.
- **Retrieving** top docs for user question.
- **Chatting** with the retrieved docs.

🔎 You can expand this by adding advanced chunking, more robust retrieval, and quality checks. Enjoy your healthy coding! 🍎